# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## 3.모델링

### (1) 모델 라이브러리 설치
---

In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 15338, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 15338 (delta 0), reused 2 (delta 0), pack-reused 15335
Receiving objects: 100% (15338/15338), 14.15 MiB | 33.46 MiB/s, done.
Resolving deltas: 100% (10524/10524), done.


In [3]:
## yolov5 폴더 requirements.txt 수정 필요
## setuptools<=64.0.2

temp_str = 'setuptools<=64.0.2\n'

f = open('/content/yolov5/requirements.txt', 'r')
f_str = f.readlines()
f.close()

f2 = open('/content/yolov5/requirements.txt', 'w')

for idx, val in enumerate(f_str) :
    if 'setuptools' in val :
        idx_v = idx
        f_str.remove(val)
        f_str.insert(idx_v, temp_str)

for val in f_str :
    f2.write(val)

f2.close() 

In [4]:
!cd yolov5; pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.6.0
    Uninstalling setuptools-67.6.0:
      Successfully uninstalled setuptools-67.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.3.1 requires setuptools>65.5.1, but you have setuptools 64.0.2 which is incompatible.


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [5]:
########################
# 이 셀부터 코드 작성하세요
########################
!python yolov5/detect.py\
    --weights '/content/drive/MyDrive/에이블스쿨/실습파일/2023.03.23_미니프로젝트_3차_(2)/aug2_v5s_20/weights/best.pt'\
    --source '/content/drive/MyDrive/에이블스쿨/실습파일/2023.03.23_미니프로젝트_3차_(2)/money_self'\
    --project '/content/detected'\
    --name 'aug2_v5s_20'\
    --conf-thres 0.75\
    --iou-thres 0.25
    # --line-thickness 2

detect: weights=['/content/drive/MyDrive/에이블스쿨/실습파일/2023.03.23_미니프로젝트_3차_(2)/aug2_v5s_20/weights/best.pt'], source=/content/drive/MyDrive/에이블스쿨/실습파일/2023.03.23_미니프로젝트_3차_(2)/money_self, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.75, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/detected, name=aug2_v5s_20, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-128-gb96f35c Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
image 1/11 /content/drive/MyDrive/에이블스쿨/실습파일/2023.03.23_미니프로젝트_3차_(2)/money_self/KakaoTalk_20230324_092701629.jpg: 640x480 1 10000, 17.7ms
image 2/11 /

In [6]:
from google.colab import files

!zip -r /content/detected_aug2_v5s_20.zip /content/detected/aug2_v5s_20

files.download(filename='/content/detected_aug2_v5s_20.zip')

  adding: content/detected/aug2_v5s_20/ (stored 0%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_08.jpg (deflated 3%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629.jpg (deflated 1%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_02.jpg (deflated 2%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_09.jpg (deflated 2%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_01.jpg (deflated 2%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_06.jpg (deflated 2%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_03.jpg (deflated 3%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_04.jpg (deflated 2%)
  adding: content/detected/aug2_v5s_20/money_video.mp4 (deflated 1%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_07.jpg (deflated 3%)
  adding: content/detected/aug2_v5s_20/KakaoTalk_20230324_092701629_05.jpg (defla

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>